In [ ]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, Lambda, MaxPooling2D
from keras import backend as K
import os
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.preprocessing.image import ImageDataGenerator
import cv2
import scipy

In [ ]:
print(keras.backend.image_data_format())

In [ ]:
root_dir = '/Users/naka345/Desktop/deeplearning/car_number/'

In [ ]:
batch_size = 32
epochs = 100
num_predictions = 20
save_dir = f'{root_dir}model/'
model_name = 'plate_detect_trained_model.h5'
load_dir_path = 'output/train'


# The data, split between train and test sets:
#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')

In [ ]:
path = root_dir + load_dir_path
import glob
print(path)
x_path = glob.glob(f'{path}/20190123/*.JPG')
y_path = glob.glob(f'{path}/20190123/*.csv')
x_path.sort()
import pandas as pd
y_t_df = pd.read_csv(y_path[0],index_col=0).sort_index()


In [ ]:
import numpy as np
x_list = [(img_to_array(load_img(x))/255.) for x in x_path]
x_train = np.array(x_list)
x_shape =  x_train.shape
print(x_shape)

In [ ]:
label = y_t_df.columns
y_train = y_t_df.values
print(y_t_df.columns)
print(y_train[0:1])

In [ ]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(

    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=15,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random zoom
    zoom_range=0.1,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)


# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
#datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
'''
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_test, y_test),
                    epochs=epochs, verbose=1, workers=4,
                    callbacks=callbacks)
'''

In [ ]:
class CustomedImageDataGenerator(ImageDataGenerator):
    label_x = ['lu_x','ru_x','ld_x','rd_x',]
    label_y = ['lu_y','ru_y','ld_y','rd_y',]
    label=['lu_x', 'lu_y', 'ru_x', 'ru_y', 'ld_x', 'ld_y', 'rd_x', 'rd_y']
    vf_corres = [(0,2),(1,3),(4,6),(5,7)]
    flip_rate = 0.5
    
    def flow(self, x_data, y_data):
        self.x_ttt = x_data[0:10]
        self.y_ttt = y_data[0:10]
        #t_im=self.x_ttt[0].transpose((1,2,0))*255
        #cv2.imwrite("./tt.jpg",t_im)
        
        width_shift_range = self.width_shift_range
        height_shift_range = self.height_shift_range
        rotation_range = self.rotation_range
        zoom_range = self.zoom_range
        horizontal_flip = self.horizontal_flip
        vertical_flip = self.vertical_flip
        
        super().__init__()
        X_batch = super().flow(self.x_ttt, y=self.y_ttt)
        
        print(dir(X_batch.image_data_generator))
        X_data = X_batch[0][0]
        y_data = X_batch[0][1]

        data_length = X_data.shape[0]
        batch_size = X_batch.batch_size
        
        if data_length < batch_size:
            batch_size = data_length
        
        indices = np.random.choice(batch_size, int(batch_size*self.flip_rate), replace=False)
        print(indices)
        
        
        if horizontal_flip:
            print("horizontal_flip")
            X_data[indices] = X_data[indices, :, :, ::-1]
            y_data = self.xy_horizontal_flip(indices, y_data)
            
        if rotation_range != 0:
            print('rotation_range')
            
        if width_shift_range != 0:
            print('width_shift_range')
            
        if height_shift_range != 0:
            print('height_shift_range')
            
        if zoom_range != 0:
            if isinstance(zoom_range,float):
                zoom_range_list = [zoom_range * -1, zoom_range]
                zoom_batch_x, zoom_batch_y = self.xy_zoom_range(zoom_range_list, batch_size)
            else:
                zoom_batch_x, zoom_batch_y = self.xy_zoom_range(zoom_range, batch_size)
        
        if vertical_flip:
            print('vertical_flip')
            
        #return X_batch
        
        x_generated = np.zeros((1,3,200,200))
        y_generated = np.zeros((1,8))
        print(zoom_batch_x)
        print(zoom_batch_y)
        for x,y,zx,zy in zip(X_data, y_data, zoom_batch_x, zoom_batch_y):
            x,y = apply_affine_transform(x,y, zx=zx, zy=zy, channel_axis=0,
                               fill_mode="nearest", cval=0,
                               order=1)
            
            x = x.reshape(1,3,200,200)
            x_generated = np.vstack((x_generated, x))
            y_generated = np.vstack((y_generated, y))
        print(f"y_generated:{y_generated}")
        return x_generated[1:], y_generated[1:]
    
    def xy_horizontal_flip(self, indices, y_data):
        y_data[indices, ::2] =1- y_data[indices, ::2]
        for a,b in self.vf_corres:
            y_data[indices,a], y_data[indices,b] = (y_data[indices,b] ,y_data[indices,a])
        return y_data
    
    def xy_zoom_range(self,zoom_range,batch_size):
        if len(zoom_range) != 2:
            raise ValueError('`zoom_range` should be a tuple or list of two'
                         ' floats. Received: %s' % (zoom_range,))
        
        batch_zx = []
        batch_zy = []
        for i in range(batch_size):
            if zoom_range[0] == 1 and zoom_range[1] == 1:
                zx, zy = 1, 1
            else:
                zx, zy = np.random.uniform(zoom_range[0], zoom_range[1], 2)
            batch_zx.append(zx)
            batch_zy.append(zy)
            
        return batch_zx, batch_zy
        
        
        

test_gen = CustomedImageDataGenerator(
    # randomly rotate images in the range (deg 0 to 180)
    #rotation_range=15,
    # randomly shift images horizontally
    #width_shift_range=0.1,
    # randomly shift images vertically
    #height_shift_range=0.1,
    # set range for random zoom
    zoom_range=0.5,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0
)
print(y_train[0:1])
xx,yy=test_gen.flow(x_train[:1],y_train[:1])
print(yy)
xx = (xx*255)
xx= xx.astype(np.int16)
#yy = yy*200.
for i,x in enumerate(xx):
    x=x.transpose((1,2,0))
    cv2.imwrite(f"./test{i}.jpg",x)
    check_y_move(f"./test{i}.jpg", yy[i])


In [ ]:
160*1.08824531-8.86865386

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4*2))
#model.add(Activation('softmax'))


In [ ]:
from keras.optimizers import Adam
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])

In [ ]:
print(x_train.shape[0])
model.fit_generator(test_gen.flow(x_train[0:2], y_train[0:2]),
                             steps_per_epoch=x_train.shape[0],
                            nb_epoch=10
                   )
                             #validation_data=(X_val, y_val))

In [ ]:
print(x_train[:1].shape)
print(y_train[:1].shape)
for x_batch, y_batch in datagen.flow(x_train[:1], y_train[:1], batch_size=1):
    print(x_batch)
    print(y_batch)
    #print('[[0.68       0.7862069  0.8        0.7816092  0.68275862 0.88735632 0.8        0.88505747]]')
    break

In [ ]:
def apply_affine_transform(x, y, theta=0, tx=0, ty=0, shear=0, zx=1, zy=1,
                           row_axis=1, col_axis=2, channel_axis=2,
                           fill_mode='nearest', cval=0., order=1):
    """Applies an affine transformation specified by the parameters given.
    # Arguments
        x: 2D numpy array, single image.
        theta: Rotation angle in degrees.
        tx: Width shift.
        ty: Heigh shift.
        shear: Shear angle in degrees.
        zx: Zoom in x direction.
        zy: Zoom in y direction
        row_axis: Index of axis for rows in the input image.
        col_axis: Index of axis for columns in the input image.
        channel_axis: Index of axis for channels in the input image.
        fill_mode: Points outside the boundaries of the input
            are filled according to the given mode
            (one of `{'constant', 'nearest', 'reflect', 'wrap'}`).
        cval: Value used for points outside the boundaries
            of the input if `mode='constant'`.
        order int: order of interpolation
    # Returns
        The transformed version of the input.
    """
    if scipy is None:
        raise ImportError('Image transformations require SciPy. '
                          'Install SciPy.')
    transform_matrix = None
    if theta != 0:
        theta = np.deg2rad(theta)
        rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                    [np.sin(theta), np.cos(theta), 0],
                                    [0, 0, 1]])
        transform_matrix = rotation_matrix

    if tx != 0 or ty != 0:
        shift_matrix = np.array([[1, 0, tx],
                                 [0, 1, ty],
                                 [0, 0, 1]])
        if transform_matrix is None:
            transform_matrix = shift_matrix
        else:
            transform_matrix = np.dot(transform_matrix, shift_matrix)

    if shear != 0:
        shear = np.deg2rad(shear)
        shear_matrix = np.array([[1, -np.sin(shear), 0],
                                 [0, np.cos(shear), 0],
                                 [0, 0, 1]])
        if transform_matrix is None:
            transform_matrix = shear_matrix
        else:
            transform_matrix = np.dot(transform_matrix, shear_matrix)

    if zx != 1 or zy != 1:
        zoom_matrix = np.array([[zx, 0, 0],
                                [0, zy, 0],
                                [0, 0, 1]])
        if transform_matrix is None:
            transform_matrix = zoom_matrix
        else:
            transform_matrix = np.dot(transform_matrix, zoom_matrix)
            
    if transform_matrix is not None:
        h, w = x.shape[row_axis], x.shape[col_axis]
        '''
        y_transform_matrix = transform_matrix_offset_center(
            transform_matrix, h,w)
        
        transform_matrix = transform_matrix_offset_center(
            transform_matrix, h, w)
        '''

        x = np.rollaxis(x, channel_axis, 0)
        final_affine_matrix = transform_matrix[:2, :2]
        final_offset = transform_matrix[:2, 2]
        
        print(f'final_affine_matrix {final_affine_matrix}')
        print(f'final_offset {final_offset}')
        channel_images = [scipy.ndimage.interpolation.affine_transform(
            x_channel,
            final_affine_matrix,
            final_offset,
            order=order,
            mode=fill_mode,
            cval=cval) for x_channel in x]

        y = transform_y_label(transform_matrix, y)
        
        x = np.stack(channel_images, axis=0)
        x = np.rollaxis(x, 0, channel_axis + 1)
    return x,y

def transform_y_label(matrix, y):
        y_x = y[::2]
        y_y = y[1::2]
        y_z = np.ones(4)
        y = np.array([y_x, y_y,y_z])
        print(y)
        #matrix[:2,2] = matrix[:2, 2]/200
        print(matrix)
        matrix.astype(np.float32)
        y.astype(np.float32)
        y_all =  np.dot(matrix, y)

        return_y = np.arange(8, dtype='float32')

        for i in range(y_all[0].shape[0]):
            return_y[i*2] = y_all[0][i]
            return_y[(i*2)+1] = y_all[1][i]
        print(return_y)
        return return_y

def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    return transform_matrix

def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    transform_matrix[:2, 2] = transform_matrix[:2, 2] *
    return transform_matrix

In [ ]:
def check_y_move(image_path, y_label):
    im = cv2.imread(image_path)

    cv2.drawMarker(im,(int(y_label[0]),int(y_label[1])),(255,0,0), markerType=cv2.MARKER_STAR)
    cv2.drawMarker(im,(int(y_label[2]),int(y_label[3])),(0,255,0), markerType=cv2.MARKER_STAR)
    cv2.drawMarker(im,(int(y_label[4]),int(y_label[5])),(0,0,255), markerType=cv2.MARKER_STAR)
    cv2.drawMarker(im,(int(y_label[6]),int(y_label[7])),(0,0,0), markerType=cv2.MARKER_STAR)
    cv2.imwrite(image_path,im)

In [ ]:
np.reciprocal(2.)